# Lab 6: Building an AI agent with Strands Agents SDK

## Use Case: Language translation

In this notebook, we will focus on how to build a Strands Agent by using our fine-tuned Nova micro model

![image.png](./images/agent-arch.png)

***

## Prerequisites

In [ ]:
! pip install -U "strands-agents==1.18.0" \
    "strands-agents-tools>=0.2.16" \
    "strands-agents-builder==0.1.10"

***

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
sagemaker_session_bucket = None

if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
bucket_name = sess.default_bucket()
default_prefix = sess.default_bucket_prefix

# Initialize the Bedrock client
bedrock = boto3.client("bedrock", region_name=sess.boto_region_name)

print(f"sagemaker session region: {sess.boto_region_name}")

### Utility function

This function is used to create the agent and the tool

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel

In [ ]:
def get_bedrock_resources():
    deployments = bedrock.list_custom_model_deployments()
    model_id = deployments['modelDeploymentSummaries'][0]['customModelDeploymentArn'] if deployments['modelDeploymentSummaries'] else None
    
    guardrails = bedrock.list_guardrails()
    guardrail_id = guardrails['guardrails'][0]['id'] if guardrails['guardrails'] else None
    
    return model_id, guardrail_id

@tool
def cultural_context_tool(topic: str) -> str:
    """
    Provides cultural context and explanations about topics.
    Args:
        topic: Topic to explain culturally
    Returns:
        Cultural context and explanation
    """
    return f"Explain the cultural context and significance of: {topic}"


def create_agent(system_prompt, model_id, guardrail_id=None, max_tokens=None):
    """Create a fresh agent instance for each query"""
    model_kwargs = {
        "model_id": model_id,
        "temperature": 0.0
    }

    if max_tokens:
        model_kwargs["max_tokens"] = max_tokens

    if guardrail_id:
        model_kwargs["guardrail_id"] = guardrail_id
        model_kwargs["guardrail_version"] = "1"

    model = BedrockModel(**model_kwargs)

    agent = Agent(model=model, tools=[cultural_context_tool])
    agent.system_prompt = system_prompt

    return agent

In [ ]:
model_id, guardrail_id = get_bedrock_resources()

***

## Interact with tools

Define the System Prompt

In [ ]:
system_prompt = """
You are an AI assistant that thinks in {language} but responds in English.

CRITICAL INSTRUCTIONS:
1. ALL reasoning inside <thinking>...</thinking> tags MUST be written entirely in {language}
2. Do NOT use English in your thinking process
3. Think completely in {language}, then translate your final conclusion to English
4. You MUST use the available tools

Format:
<thinking>
[Your complete reasoning process in {language} only]
</thinking>

Answer
"""

#### Query 1

In [ ]:
language = "French"

query = """
Tell me about French cuisine
"""

In [ ]:
agent = create_agent(
    system_prompt.format(language=language), model_id, guardrail_id, max_tokens=8192
)

response = agent(query)

#### Query 2

In [ ]:
language = "Spanish"

query = """
What is the cultural significance of flamenco?
"""

In [ ]:
agent = create_agent(
    system_prompt.format(language=language), model_id, guardrail_id, max_tokens=8192
)

response = agent(query)

#### Query 3

In [ ]:
language = "German"

query = """
Tell me about Oktoberfest
"""

In [ ]:
agent = create_agent(
    system_prompt.format(language=language), model_id, guardrail_id, max_tokens=8192
)

response = agent(query)

#### Query 4

In [ ]:
language = "Italian"

query = """
Talk about Dante's Divina Commedia
"""

In [ ]:
agent = create_agent(
    system_prompt.format(language=language), model_id, guardrail_id, max_tokens=8192
)

response = agent(query)